Récupérer les données d'un joueur :

faire des features sur les derniers matchs joués en temps 100 jours.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

from datetime import datetime, timedelta 
from sklearn.model_selection import train_test_split

pd.set_option('max_columns', 25)

df = pd.read_csv("atp_data.csv")

In [2]:
df.Date = pd.to_datetime(df.Date)
date = pd.DataFrame(df.Date)

In [3]:
df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,Wsets,Lsets,Comment,PSW,PSL,B365W,B365L,elo_winner,elo_loser,proba_elo
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63,77,2.0,0.0,Completed,NaN,NaN,NaN,NaN,1500.0,1500.0,0.5
1,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Kiefer N.,Tarango J.,6,59,2.0,0.0,Completed,NaN,NaN,NaN,NaN,1500.0,1500.0,0.5
2,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Gaudio G.,Luxa P.,73,174,2.0,1.0,Completed,NaN,NaN,NaN,NaN,1500.0,1500.0,0.5
3,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,El Aynaoui Y.,Dupuis A.,33,78,2.0,1.0,Completed,NaN,NaN,NaN,NaN,1500.0,1500.0,0.5
4,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Cherkasov A.,Arazi H.,206,35,2.0,0.0,Completed,NaN,NaN,NaN,NaN,1500.0,1500.0,0.5


In [4]:
#récupérer les matchs effectués par le joueur A 150 jours avant son match i
def match_150_avant(i):
    #on obtient le match qui se prépare
    match = df.iloc[i,:]
    #on garde les matchs précédents uniquement.
    match_avant = df.iloc[:i-1,:]
    joueurA = match_avant[(match_avant["Winner"] == match.Winner) | (match_avant["Loser"] == match.Winner)]
    joueurB = match_avant[(match_avant["Winner"] == match.Loser) | (match_avant["Loser"]== match.Loser)]
    joueurA = joueurA[(joueurA.Date <= match.Date) & (joueurA.Date >= (match.Date - timedelta(days=150)))]
    joueurB = joueurB[(joueurB.Date <= match.Date) & (joueurB.Date >= (match.Date - timedelta(days=150)))]
    
    #ajouter le taux de victoire des joueurs sur les 150 derniers jours
    if (len(joueurA)== 0):
        df.loc[i,"ratio_victoire_A"] = "NaN"
    else:
        df.loc[i,"ratio_victoire_A"] = len(joueurA[joueurA["Winner"]== match.Winner])/len(joueurA)
    if (len(joueurB)== 0):    
        df.loc[i,"ratio_victoire_B"] = "Nan"
    else:
        df.loc[i,"ratio_victoire_B"] = len(joueurB[joueurB["Winner"]== match.Loser])/len(joueurB)
    
    
    #progression elo points sur cette période
    df.loc[i,"progression_elo_A"]=  max( joueurA.elo_winner.max(),joueurA.elo_loser.max() ) - min( joueurA.elo_winner.min(),joueurA.elo_loser.min() )
    df.loc[i,"progression_elo_B"]=  max( joueurB.elo_winner.max(),joueurB.elo_loser.max() ) - min( joueurB.elo_winner.min(),joueurB.elo_loser.min() )
    
    
    #ajouter le taux de victoire sur la surface en question
    surface_A = joueurA[joueurA["Surface"]== match.Surface]
    surface_B = joueurB[joueurB["Surface"]== match.Surface]
    if (len(surface_A)== 0):
        df.loc[i,"surface_victoire_A"] = "NaN"
    else:
        df.loc[i,"surface_victoire_A"] = len(surface_A[surface_A["Winner"]== match.Winner])/len(surface_A)
    if (len(surface_B)== 0):    
        df.loc[i,"surface_victoire_B"] = "NaN"
    else:
        df.loc[i,"surface_victoire_B"] = len(surface_B[surface_B["Winner"]== match.Loser])/len(surface_B)
    
    #ajouter le taux de victoire selon les conditions exterieur ou intérieur
    court_A = joueurA[joueurA["Court"]== match.Court]
    court_B = joueurB[joueurB["Court"]== match.Court]
    if (len(court_A)== 0):
        df.loc[i,"court_victoire_A"] = "NaN"
    else:
        df.loc[i,"court_victoire_A"] = len(court_A[court_A["Winner"]== match.Winner])/len(court_A)
    if (len(court_B)== 0):    
        df.loc[i,"court_victoire_B"] = "NaN"
    else:
        df.loc[i,"court_victoire_B"] = len(court_B[court_B["Winner"]== match.Loser])/len(court_B)
    
    
    #nombre de sets joués sur le tournoi en question
    set_A = joueurA[joueurA["Tournament"]== match.Tournament]
    set_B = joueurB[joueurB["Tournament"]== match.Tournament]
    df.loc[i,"sets_A"] = set_A.Wsets.sum()+set_A.Lsets.sum()
    df.loc[i,"sets_B"] = set_B.Wsets.sum()+set_B.Lsets.sum()
    
    

In [5]:
for i in range (len(df)):
    match_150_avant(i)

In [6]:
df

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,...,elo_loser,proba_elo,ratio_victoire_A,ratio_victoire_B,progression_elo_A,progression_elo_B,surface_victoire_A,surface_victoire_B,court_victoire_A,court_victoire_B,sets_A,sets_B
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63,...,1500.000000,0.500000,0.5,0,16.000000,0.000000,0.5,0,0.5,0,4.0,2.0
1,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Kiefer N.,Tarango J.,6,...,1500.000000,0.500000,NaN,Nan,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Gaudio G.,Luxa P.,73,...,1500.000000,0.500000,NaN,Nan,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,El Aynaoui Y.,Dupuis A.,33,...,1500.000000,0.500000,NaN,Nan,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,3,Doha,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Cherkasov A.,Arazi H.,206,...,1500.000000,0.500000,NaN,Nan,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44703,17,Dubai,Dubai Tennis Championships,2018-03-03,ATP500,Outdoor,Hard,The Final,3,Bautista Agut R.,Pouille L.,23,...,1842.459807,0.490563,0.65,0.782609,549.181425,390.335084,0.65,0.782609,0.714286,0.714286,9.0,11.0
44704,16,Acapulco,Abierto Mexicano,2018-03-03,ATP500,Outdoor,Hard,Semifinals,3,Anderson K.,Donaldson J.,8,...,1647.967561,0.755103,0.631579,0.466667,419.064329,706.170958,0.631579,0.466667,0.636364,0.583333,6.0,5.0
44705,18,Sao Paulo,Brasil Open,2018-03-03,ATP250,Indoor,Clay,Semifinals,3,Jarry N.,Zeballos H.,73,...,1619.758421,0.454841,0.692308,0.5,366.971153,434.384203,0.8,0.571429,1,1,9.0,9.0
44706,16,Acapulco,Abierto Mexicano,2018-03-04,ATP500,Outdoor,Hard,The Final,3,Del Potro J.M.,Anderson K.,9,...,1851.413609,0.659862,0.8,0.684211,709.739333,419.064329,0.8,0.684211,0.777778,0.727273,9.0,9.0


In [7]:
df.to_csv("data_clean_feature_player.csv")